In [39]:
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
import pickle

import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

#Arabic library
import arabic_reshaper
from bidi.algorithm import get_display
#voice Libraries
import pyttsx3
from gtts import gTTS
import IPython.display as ipd

from io import BytesIO
import time
import pygame



In [40]:


mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [41]:
with open('body_language_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [42]:
def speake(text):
    mp3_fp =BytesIO()
    tts = gTTS(text,lang="ar")
    tts.write_to_fp(mp3_fp)
    return mp3_fp

def extract_arabic(word,labels_dict): 
    body =word
    for i in labels_dict:
        if i[0] ==body:
            print("i",i)
            return i[1]
    return 0
    

In [44]:
cap = cv2.VideoCapture(0)
# arabic words
labels_dict = [["al salam","السلام"],["alsalam","السلام"],["alekom","عليكم"],["Ismy","اسمي"],["ana","أنا"]]
fontFile ="F:\\Arabic-Sign-Language-main\\fonts\\Sahel.ttf"
font = ImageFont.truetype(fontFile, 70)
arabic_text=""
old_text=""
# voice
duration =2
start_time = time.time()
all_temp=""
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
     
        # Extract Pose landmarks
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
        
        # Extract Face landmarks
        face = results.face_landmarks.landmark
        face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
    
        # Concate rows
        row = pose_row+face_row
        
        
#             # Append class name 
#             row.insert(0, class_name)
        
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

        # Make Detections
        X = pd.DataFrame([row])
        body_language_class = model.predict(X)[0]
        print("body_language_class",body_language_class)
        body_language_prob = model.predict_proba(X)[0]
        print(body_language_class, body_language_prob)
        
        #arabic text
        
        sequence = extract_arabic(body_language_class,labels_dict)
        print("seq",sequence)
        reshaped_text = arabic_reshaper.reshape(sequence)   
        print("reshaped_text",reshaped_text)
        bidi_text = get_display(reshaped_text)
        print("bidi_text",bidi_text)
        
        img_pil = Image.fromarray(frame)
        draw = ImageDraw.Draw(img_pil)
        draw.text((97,15), bidi_text, (0,0,0), font=font)
        frame = np.array(img_pil)
        
        #Arabic voice
        
        current_time = time.time()
        elapsed_time = current_time - start_time
        print("elapsed_time",elapsed_time)
        if reshaped_text==old_text and elapsed_time >= duration:
            pygame.init()
            pygame.mixer.init()
            sound=speake(reshaped_text)
            sound.seek(0)
            pygame.mixer.music.load(sound,"mp3")
            pygame.mixer.music.play()
            temp=reshaped_text   
            arabic_text=  arabic_text+" "+temp
            print("arabic,text",arabic_text)
            start_time = time.time()
        old_text=reshaped_text
        print("old_text",old_text)
        
        
        
        # Grab ear coords
        coords = tuple(np.multiply(
                        np.array(
                            (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                    , [640,480]).astype(int))
        
        cv2.rectangle(image, 
                        (coords[0], coords[1]+5), 
                        (coords[0]+len(body_language_class)*20, coords[1]-30), 
                        (245, 117, 16), -1)
        cv2.putText(image, body_language_class, coords, 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Get status box
        cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
        
        # Display Class
        
        cv2.putText(image, 'CLASS'
                    , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, body_language_class.split(' ')[0]
                    , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        # Display Probability
        
        cv2.putText(image, 'PROB'
                    , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                    , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
   
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


body_language_class Ismy
Ismy [0.46 0.01 0.37 0.01 0.15]
i ['Ismy', 'اسمي']
seq اسمي
reshaped_text ﺍﺳﻤﻲ
bidi_text ﻲﻤﺳﺍ
elapsed_time 2.8730292320251465
old_text ﺍﺳﻤﻲ


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


body_language_class alekom
alekom [0.33 0.01 0.42 0.04 0.2 ]
i ['alekom', 'عليكم']
seq عليكم
reshaped_text ﻋﻠﻴﻜﻢ
bidi_text ﻢﻜﻴﻠﻋ
elapsed_time 3.975788116455078
old_text ﻋﻠﻴﻜﻢ


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

body_language_class alekom
alekom [0.24 0.06 0.43 0.02 0.25]
i ['alekom', 'عليكم']
seq عليكم
reshaped_text ﻋﻠﻴﻜﻢ
bidi_text ﻢﻜﻴﻠﻋ
elapsed_time 2.5819406509399414
old_text ﻋﻠﻴﻜﻢ


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


body_language_class alekom
alekom [0.23 0.06 0.52 0.01 0.18]
i ['alekom', 'عليكم']
seq عليكم
reshaped_text ﻋﻠﻴﻜﻢ
bidi_text ﻢﻜﻴﻠﻋ
elapsed_time 3.0672829151153564
arabic,text  ﻋﻠﻴﻜﻢ ﺍﺳﻤﻲ ﺍﺳﻤﻲ ﺍﻟﺴﻼﻡ ﻋﻠﻴﻜﻢ ﺍﺳﻤﻲ ﺍﻟﺴﻼﻡ ﻋﻠﻴﻜﻢ ﺍﺳﻤﻲ ﻋﻠﻴﻜﻢ
old_text ﻋﻠﻴﻜﻢ


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


body_language_class alekom
alekom [0.18 0.03 0.48 0.03 0.28]
i ['alekom', 'عليكم']
seq عليكم
reshaped_text ﻋﻠﻴﻜﻢ
bidi_text ﻢﻜﻴﻠﻋ
elapsed_time 0.7775459289550781
old_text ﻋﻠﻴﻜﻢ


In [38]:
cap.release()

In [ ]:
tem="السلام عليكم أنا اسمي"

In [ ]:
print(arabic_text)

 ﺍﻟﺴﻼﻡ ﺍﺳﻤﻲ ﻋﻠﻴﻜﻢ ﺍﻟﺴﻼﻡ ﻋﻠﻴﻜﻢ ﺍﻟﺴﻼﻡ ﻋﻠﻴﻜﻢ ﻋﻠﻴﻜﻢ


In [36]:
tts = gTTS(tem, lang='ar')
tts.save('test1.mp3')
audio_path="test1.mp3"
ipd.Audio(audio_path, autoplay=True)

gTTSError: Failed to connect. Probable cause: Unknown

In [33]:
audio_path="test.mp3"
ipd.Audio(audio_path, autoplay=True)